In [1]:
# Filter EEG and label files
import os
import re
import glob
import numpy as np
data_dir = "/home/glab/Desktop/gruendemann2/Amine/full_sleep_data/"

def numerical_sort(filename):
    # Extract the number in the filename using a regular expression
    numbers = re.findall(r'(\d+)', filename)
    return [int(num) for num in numbers]

#all_files = os.listdir(data_dir)
eeg_files = sorted(glob.glob(data_dir + '*eegeegmove4model_eeg.npy'), key=numerical_sort)

label_files = sorted(glob.glob(data_dir + '*eegeegmove4model_label.npy'), key=numerical_sort)

#print(eeg_files)
#print(label_files)
###Sanity check
for file in range(len(eeg_files)):
    eeg = np.load(os.path.join(data_dir, eeg_files[file]))
    labels = np.load(os.path.join(data_dir, label_files[file]))
    print(eeg.shape, labels.shape)

(568, 9, 1280, 3) (568,)
(447, 9, 1280, 3) (447,)
(465, 9, 1280, 3) (465,)
(337, 9, 1280, 3) (337,)
(506, 9, 1280, 3) (506,)
(569, 9, 1280, 3) (569,)
(518, 9, 1280, 3) (518,)
(835, 9, 1280, 3) (835,)
(381, 9, 1280, 3) (381,)
(480, 9, 1280, 3) (480,)
(571, 9, 1280, 3) (571,)
(509, 9, 1280, 3) (509,)
(473, 9, 1280, 3) (473,)
(513, 9, 1280, 3) (513,)
(445, 9, 1280, 3) (445,)
(380, 9, 1280, 3) (380,)
(325, 9, 1280, 3) (325,)
(349, 9, 1280, 3) (349,)
(271, 9, 1280, 3) (271,)
(294, 9, 1280, 3) (294,)
(459, 9, 1280, 3) (459,)
(416, 9, 1280, 3) (416,)
(378, 9, 1280, 3) (378,)
(285, 9, 1280, 3) (285,)
(255, 9, 1280, 3) (255,)
(357, 9, 1280, 3) (357,)
(303, 9, 1280, 3) (303,)
(568, 9, 1280, 3) (568,)
(651, 9, 1280, 3) (651,)
(551, 9, 1280, 3) (551,)
(384, 9, 1280, 3) (384,)
(499, 9, 1280, 3) (499,)
(410, 9, 1280, 3) (410,)
(648, 9, 1280, 3) (648,)
(495, 9, 1280, 3) (495,)
(615, 9, 1280, 3) (615,)
(348, 9, 1280, 3) (348,)
(550, 9, 1280, 3) (550,)
(513, 9, 1280, 3) (513,)
(473, 9, 1280, 3) (473,)


### Data Normalization

In [2]:
import os
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report
from utils import z_score_normalize# For classification report
train_eeg_data = None
train_labels = None
val_eeg_data = None
val_labels = None
# Set device to GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

test_index=22
data_dir = "/home/glab/Desktop/gruendemann2/Amine/full_sleep_data/"

# List all files in the directory

for file in range(len(eeg_files)):
        if 1 != 2:  # Exclude the current test index
            # Load EEG data
            eeg = np.load(os.path.join(data_dir, eeg_files[file]))
            
            # Exclude the first 8 blocks and keep only the last block (block 9)
            eeg = eeg[:, -1, :, :]  # Shape: (n_samples, 1280, 3)
            
            # Exclude the third channel (keep only the first two channels)
            eeg = eeg[:, :, :2]  # Shape: (n_samples, 1280, 2)
            
            
            # Load labels
            label = np.load(os.path.join(data_dir, label_files[file]))
            label[label == 3] = 2
            
            
            assert eeg.shape[0] == label.shape[0], f"Mismatch in number of samples between EEG data and labels for file {eeg_files[file]} and {label_files[file]}: " \
                                               f"{eeg.shape[0]} samples in EEG, {label.shape[0]} samples in labels."

            print(eeg.shape, label.shape)
            for channel in range(eeg.shape[2]):  # Iterate over channels
                eeg[:, :, channel] = z_score_normalize(eeg[:, :, channel])
            # Stack EEG data under each other
            if train_eeg_data is None:
                train_eeg_data = eeg
                
            else:
                train_eeg_data = np.vstack((train_eeg_data, eeg))

            # Stack labels under each other
            if train_labels is None:
                train_labels = label
            else:
                train_labels = np.hstack((train_labels, label))



Using device: cuda
(568, 1280, 2) (568,)
(447, 1280, 2) (447,)
(465, 1280, 2) (465,)
(337, 1280, 2) (337,)
(506, 1280, 2) (506,)
(569, 1280, 2) (569,)
(518, 1280, 2) (518,)
(835, 1280, 2) (835,)
(381, 1280, 2) (381,)
(480, 1280, 2) (480,)
(571, 1280, 2) (571,)
(509, 1280, 2) (509,)
(473, 1280, 2) (473,)
(513, 1280, 2) (513,)
(445, 1280, 2) (445,)
(380, 1280, 2) (380,)
(325, 1280, 2) (325,)
(349, 1280, 2) (349,)
(271, 1280, 2) (271,)
(294, 1280, 2) (294,)
(459, 1280, 2) (459,)
(416, 1280, 2) (416,)
(378, 1280, 2) (378,)
(285, 1280, 2) (285,)
(255, 1280, 2) (255,)
(357, 1280, 2) (357,)
(303, 1280, 2) (303,)
(568, 1280, 2) (568,)
(651, 1280, 2) (651,)
(551, 1280, 2) (551,)
(384, 1280, 2) (384,)
(499, 1280, 2) (499,)
(410, 1280, 2) (410,)
(648, 1280, 2) (648,)
(495, 1280, 2) (495,)
(615, 1280, 2) (615,)
(348, 1280, 2) (348,)
(550, 1280, 2) (550,)
(513, 1280, 2) (513,)
(473, 1280, 2) (473,)
(494, 1280, 2) (494,)
(571, 1280, 2) (571,)
(524, 1280, 2) (524,)
(365, 1280, 2) (365,)
(591, 1280, 2

In [3]:
import torch
import numpy as np
from sklearn.utils.class_weight import compute_class_weight


# Convert train_eeg_data and train_labels to PyTorch tensors
# Ensure that train_eeg_data is already in the correct format (numpy or list) before converting
train_eeg_data = torch.tensor(train_eeg_data, dtype=torch.float32).to(device)  # Convert to float32 for input data
train_labels = torch.tensor(train_labels, dtype=torch.long).to(device)  # Convert to long (int64) for labels

# Ensure both 'train_labels' and 'train_eeg_data' are numpy arrays to compute class weights
train_labels_np = train_labels.cpu().numpy()  # Convert to numpy array for sklearn
classes_np = np.unique(train_labels_np)  # Get unique classes from labels

# Print the unique classes to make sure they are correctly identified
print(f"Unique classes: {classes_np}")

# Compute class weights using sklearn, with class_weight='balanced'
class_weights = compute_class_weight(class_weight='balanced', classes=classes_np, y=train_labels_np)

# Print the class weights to verify the computed values
print(f"Class weights: {class_weights}")
label_counts = torch.bincount(train_labels)

# Print the occurrences for each class
for label, count in enumerate(label_counts):
    print(f"Class {label}: {count.item()} occurrences")

# Convert class_weights to a tensor and move to the correct device


# You can then use this criterion for training:
# outputs = model(inputs) 
# loss = criterion(outputs, labels)


Unique classes: [0 1 2]
Class weights: [0.58754096 6.83961147 0.86821845]
Class 0: 16379 occurrences
Class 1: 1407 occurrences
Class 2: 11084 occurrences


In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset, random_split
from sklearn.metrics import f1_score, precision_score, recall_score
import random
from sklearn.metrics import classification_report
import optuna
import numpy as np
import pandas as pd
import os
from eegnet import EEGNet

# ------------------ 1. Reproducibility ------------------
def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed(42)

# ------------------ 2. Device ------------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ------------------ 3. Dataset ------------------
full_dataset = TensorDataset(train_eeg_data, train_labels)
total_size = len(full_dataset)
val_size = int(0.05 * total_size)
train_size = total_size - val_size
generator = torch.Generator().manual_seed(42)

train_dataset, val_dataset = random_split(full_dataset, [train_size, val_size], generator=generator)

# ------------------ 4. Training Function ------------------
def train_and_evaluate(F1, F2, D, dropout, learning_rate, batch_size, weight_decay):
    # Initialize the model with the current parameters
    model = EEGNet(F1=F1, F2=F2, D=D, dropout_rate=dropout, num_classes=3).to(device)
    
    # Initialize optimizer with weight decay
    optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
    criterion = nn.CrossEntropyLoss()
    
    # DataLoaders with current batch size
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
    
    # Training loop
    num_epochs = 300
    patience = 30  # Early stopping patience
    best_val_loss = float('inf')
    counter = 0
    
    for epoch in range(num_epochs):
        model.train()
        epoch_loss = 0
        
        for batch_eeg, batch_labels in train_loader:
            batch_eeg, batch_labels = batch_eeg.to(device), batch_labels.to(device)
            
            # Forward pass
            outputs = model(batch_eeg)
            loss = criterion(outputs, batch_labels)
            
            # Backpropagation
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            epoch_loss += loss.item()
        
        avg_train_loss = epoch_loss / len(train_loader)
        
        # Validation phase
        model.eval()
        val_loss = 0
        
        with torch.no_grad():
            for batch_eeg, batch_labels in val_loader:
                batch_eeg, batch_labels = batch_eeg.to(device), batch_labels.to(device)
                outputs = model(batch_eeg)
                loss = criterion(outputs, batch_labels)
                val_loss += loss.item()
        
        avg_val_loss = val_loss / len(val_loader)
        
        # Print progress
        print(f"Epoch {epoch+1}/{num_epochs} - Train Loss: {avg_train_loss:.4f}, Val Loss: {avg_val_loss:.4f}")
        
        # Early stopping check
        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            counter = 0
            # Save best model based on validation loss
            torch.save(model.state_dict(), "current_best_model.pth")
        else:
            counter += 1
            if counter >= patience:
                print(f"Early stopping at epoch {epoch+1}")
                break
    
    # Load the best model before final evaluation
    model.load_state_dict(torch.load("current_best_model.pth"))
    
    # Evaluation
    model.eval()
    all_preds, all_labels = [], []
    
    with torch.no_grad():
        for batch_eeg, batch_labels in val_loader:
            batch_eeg, batch_labels = batch_eeg.to(device), batch_labels.to(device)
            outputs = model(batch_eeg)
            _, predicted = torch.max(outputs.data, 1)
            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(batch_labels.cpu().numpy())
    
    # Metrics - using macro average instead of weighted
    f1 = f1_score(all_labels, all_preds, average='macro')
    precision = precision_score(all_labels, all_preds, average='macro')
    recall = recall_score(all_labels, all_preds, average='macro')
    
    print(f"Macro F1 Score: {f1:.4f}, Macro Precision: {precision:.4f}, Macro Recall: {recall:.4f}")
    report = classification_report(all_labels, all_preds)
    print("\nClassification Report:\n", report)
    
    return {
        'f1_score': f1,
        'precision': precision,
        'recall': recall,
        'val_loss': best_val_loss
    }

# ------------------ 5. Optuna Study ------------------
def objective(trial):
    # Define hyperparameters to optimize - focusing on ranges appropriate for 2 channels
    F1 = trial.suggest_categorical('F1', [4, 8, 16, 32])
    F2 = trial.suggest_categorical('F2', [8, 16, 32])
    D = trial.suggest_categorical('D', [2, 4,8])
    dropout = trial.suggest_float('dropout', 0.1, 0.7)
    learning_rate = trial.suggest_float('learning_rate', 1e-5, 1e-3, log=True)
    batch_size = trial.suggest_categorical('batch_size', [32, 64, 128, 256])
    weight_decay = trial.suggest_float('weight_decay', 1e-5, 1e-2, log=True)
    
    print(f"\nTrial {trial.number + 1}")
    print(f"Training with F1={F1}, F2={F2}, D={D}, dropout={dropout}, LR={learning_rate}, BS={batch_size}, WD={weight_decay}")
    
    # Train and evaluate model
    metrics = train_and_evaluate(F1, F2, D, dropout, learning_rate, batch_size, weight_decay)
    
    # Store all results
    trial.set_user_attr('precision', metrics['precision'])
    trial.set_user_attr('recall', metrics['recall'])
    trial.set_user_attr('val_loss', metrics['val_loss'])
    
    return metrics['f1_score']  # Optimize for F1 score

# ------------------ 6. Run Optimization ------------------
best_f1 = 0
best_params = {}
results = []
save_every_n = 5  # Save intermediate results every n trials

# Create a study object and optimize the objective function
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=500)  # Adjust number of trials as needed

# ------------------ 7. Save Results ------------------
# Extract and save all trials' results
for trial in study.trials:
    trial_params = trial.params
    results.append({
        'F1': trial_params.get('F1'),
        'F2': trial_params.get('F2'),
        'D': trial_params.get('D'),
        'dropout': trial_params.get('dropout'),
        'learning_rate': trial_params.get('learning_rate'),
        'batch_size': trial_params.get('batch_size'),
        'weight_decay': trial_params.get('weight_decay'),
        'f1_score': trial.value,
        'precision': trial.user_attrs.get('precision'),
        'recall': trial.user_attrs.get('recall'),
        'val_loss': trial.user_attrs.get('val_loss')
    })
    
    # Save model with best F1 score
    if trial.value > best_f1:
        best_f1 = trial.value
        best_params = trial_params
        print(f"*** New best model found with F1: {best_f1:.4f} ***")

# Save all results to CSV
results_df = pd.DataFrame(results)
results_df.to_csv("hyperparameter_results_optuna.csv", index=False)
print("\nAll results saved to 'hyperparameter_results_optuna.csv'")

# Print best parameters
print(f"\nBest F1 Score: {best_f1:.4f}")
print(f"Best Hyperparameters: {best_params}")

# You can get more insights from the study
print("\nStudy statistics: ")
print(f"  Number of finished trials: {len(study.trials)}")
print(f"  Best trial: {study.best_trial.number}")

# Plot optimization history
try:
    from optuna.visualization import plot_optimization_history
    fig = plot_optimization_history(study)
    fig.write_html("optuna_optimization_history.html")
    print("Optimization history plot saved to 'optuna_optimization_history.html'")
except ImportError:
    print("Optuna visualization requires plotly. Install with: pip install plotly")

[I 2025-04-30 22:20:13,381] A new study created in memory with name: no-name-1db7fda6-b9da-4e91-a325-713fbfaeaeb6



Trial 1
Training with F1=8, F2=16, D=2, dropout=0.634131042579391, LR=2.5471659095775814e-05, BS=128, WD=0.0003030088405181484
Epoch 1/300 - Train Loss: 0.9987, Val Loss: 0.9704
Epoch 2/300 - Train Loss: 0.8394, Val Loss: 0.8514
Epoch 3/300 - Train Loss: 0.7257, Val Loss: 0.7243
Epoch 4/300 - Train Loss: 0.6039, Val Loss: 0.5786
Epoch 5/300 - Train Loss: 0.4864, Val Loss: 0.4522
Epoch 6/300 - Train Loss: 0.3994, Val Loss: 0.3651
Epoch 7/300 - Train Loss: 0.3362, Val Loss: 0.3167
Epoch 8/300 - Train Loss: 0.2871, Val Loss: 0.2623
Epoch 9/300 - Train Loss: 0.2558, Val Loss: 0.2385
Epoch 10/300 - Train Loss: 0.2318, Val Loss: 0.2120
Epoch 11/300 - Train Loss: 0.2145, Val Loss: 0.2019
Epoch 12/300 - Train Loss: 0.2031, Val Loss: 0.1825
Epoch 13/300 - Train Loss: 0.1884, Val Loss: 0.1775
Epoch 14/300 - Train Loss: 0.1849, Val Loss: 0.1673
Epoch 15/300 - Train Loss: 0.1774, Val Loss: 0.1699
Epoch 16/300 - Train Loss: 0.1723, Val Loss: 0.1607
Epoch 17/300 - Train Loss: 0.1721, Val Loss: 0.16

/tmp/ipykernel_524047/1455158977.py:105: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("current_best_model.pth"))
[I 2025-04-30 22:22:41,567

Epoch 153/300 - Train Loss: 0.1044, Val Loss: 0.0940
Early stopping at epoch 153
Macro F1 Score: 0.9474, Macro Precision: 0.9336, Macro Recall: 0.9638

Classification Report:
               precision    recall  f1-score   support

           0       0.97      0.99      0.98       789
           1       0.84      0.95      0.89        61
           2       0.99      0.95      0.97       593

    accuracy                           0.97      1443
   macro avg       0.93      0.96      0.95      1443
weighted avg       0.97      0.97      0.97      1443


Trial 2
Training with F1=8, F2=8, D=4, dropout=0.18463296029798004, LR=1.359807186908951e-05, BS=64, WD=0.00014318798607877405
Epoch 1/300 - Train Loss: 1.0069, Val Loss: 0.9172
Epoch 2/300 - Train Loss: 0.7949, Val Loss: 0.6739
Epoch 3/300 - Train Loss: 0.5673, Val Loss: 0.4810
Epoch 4/300 - Train Loss: 0.4412, Val Loss: 0.3941
Epoch 5/300 - Train Loss: 0.3764, Val Loss: 0.3445
Epoch 6/300 - Train Loss: 0.3317, Val Loss: 0.3019
Epoch 7/3

Epoch 145/300 - Train Loss: 0.0853, Val Loss: 0.0772
Epoch 146/300 - Train Loss: 0.0829, Val Loss: 0.0789
Epoch 147/300 - Train Loss: 0.0837, Val Loss: 0.0775
Epoch 148/300 - Train Loss: 0.0836, Val Loss: 0.0797
Epoch 149/300 - Train Loss: 0.0822, Val Loss: 0.0773
Epoch 150/300 - Train Loss: 0.0821, Val Loss: 0.0779
Epoch 151/300 - Train Loss: 0.0831, Val Loss: 0.0765
Epoch 152/300 - Train Loss: 0.0818, Val Loss: 0.0798
Epoch 153/300 - Train Loss: 0.0830, Val Loss: 0.0763
Epoch 154/300 - Train Loss: 0.0833, Val Loss: 0.0780
Epoch 155/300 - Train Loss: 0.0828, Val Loss: 0.0781
Epoch 156/300 - Train Loss: 0.0829, Val Loss: 0.0773
Epoch 157/300 - Train Loss: 0.0818, Val Loss: 0.0768
Epoch 158/300 - Train Loss: 0.0829, Val Loss: 0.0759
Epoch 159/300 - Train Loss: 0.0821, Val Loss: 0.0765
Epoch 160/300 - Train Loss: 0.0830, Val Loss: 0.0790
Epoch 161/300 - Train Loss: 0.0810, Val Loss: 0.0774
Epoch 162/300 - Train Loss: 0.0826, Val Loss: 0.0768
Epoch 163/300 - Train Loss: 0.0809, Val Loss: 

...

Epoch 78/300 - Train Loss: 0.0510, Val Loss: 0.0709
Early stopping at epoch 78
Macro F1 Score: 0.9710, Macro Precision: 0.9692, Macro Recall: 0.9728

Classification Report:
               precision    recall  f1-score   support

           0       0.98      0.99      0.99       789
           1       0.94      0.95      0.94        61
           2       0.99      0.98      0.98       593

    accuracy                           0.98      1443
   macro avg       0.97      0.97      0.97      1443
weighted avg       0.98      0.98      0.98      1443


Trial 499
Training with F1=32, F2=8, D=8, dropout=0.23497913386635555, LR=0.00022079295462288558, BS=32, WD=5.1806684276364865e-05
Epoch 1/300 - Train Loss: 0.2290, Val Loss: 0.0969
Epoch 2/300 - Train Loss: 0.1069, Val Loss: 0.0764
Epoch 3/300 - Train Loss: 0.0970, Val Loss: 0.0708
Epoch 4/300 - Train Loss: 0.0919, Val Loss: 0.0943
Epoch 5/300 - Train Loss: 0.0886, Val Loss: 0.0731
Epoch 6/300 - Train Loss: 0.0861, Val Loss: 0.0762
Epoch 7

/tmp/ipykernel_524047/1455158977.py:105: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("current_best_model.pth"))
[I 2025-05-02 05:45:16,918

Epoch 71/300 - Train Loss: 0.0642, Val Loss: 0.0644
Early stopping at epoch 71
Macro F1 Score: 0.9698, Macro Precision: 0.9640, Macro Recall: 0.9761

Classification Report:
               precision    recall  f1-score   support

           0       0.98      0.99      0.98       789
           1       0.92      0.97      0.94        61
           2       0.99      0.97      0.98       593

    accuracy                           0.98      1443
   macro avg       0.96      0.98      0.97      1443
weighted avg       0.98      0.98      0.98      1443


Trial 500
Training with F1=16, F2=32, D=2, dropout=0.12127149797542267, LR=7.019918163943999e-05, BS=32, WD=3.7133046010277204e-05
Epoch 1/300 - Train Loss: 0.3539, Val Loss: 0.1482
Epoch 2/300 - Train Loss: 0.1423, Val Loss: 0.1023
Epoch 3/300 - Train Loss: 0.1170, Val Loss: 0.0882
Epoch 4/300 - Train Loss: 0.1076, Val Loss: 0.0931
Epoch 5/300 - Train Loss: 0.0997, Val Loss: 0.0776
Epoch 6/300 - Train Loss: 0.0967, Val Loss: 0.0855
Epoch 7

/tmp/ipykernel_524047/1455158977.py:105: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("current_best_model.pth"))
[I 2025-05-02 05:47:07,994

Epoch 52/300 - Train Loss: 0.0695, Val Loss: 0.0720
Early stopping at epoch 52
Macro F1 Score: 0.9670, Macro Precision: 0.9593, Macro Recall: 0.9756

Classification Report:
               precision    recall  f1-score   support

           0       0.98      0.99      0.98       789
           1       0.91      0.97      0.94        61
           2       0.99      0.97      0.98       593

    accuracy                           0.98      1443
   macro avg       0.96      0.98      0.97      1443
weighted avg       0.98      0.98      0.98      1443

*** New best model found with F1: 0.9474 ***
*** New best model found with F1: 0.9667 ***
*** New best model found with F1: 0.9680 ***
*** New best model found with F1: 0.9694 ***
*** New best model found with F1: 0.9700 ***
*** New best model found with F1: 0.9720 ***
*** New best model found with F1: 0.9743 ***
*** New best model found with F1: 0.9761 ***
*** New best model found with F1: 0.9790 ***
*** New best model found with F1: 0.9804